# Map view of all food recognized processing points in Nigeria
---

In [1]:
import pandas as pd
import geopandas as gpd
import folium

In [2]:
aqua_data    = gpd.read_file("data/commodity_processing_pts/Aquaculture.shp")
cassava_data = gpd.read_file("data/commodity_processing_pts/Cassava.shp")
cocoa_data   = gpd.read_file("data/commodity_processing_pts/Cocoa.shp")
cotton_data  = gpd.read_file("data/commodity_processing_pts/Cotton.shp")
oil_data     = gpd.read_file("data/commodity_processing_pts/Palm_oil.shp")
rice_data    = gpd.read_file("data/commodity_processing_pts/Rice.shp")
sorghum_data = gpd.read_file("data/commodity_processing_pts/Sorghum.shp")

processing_pts = pd.concat([aqua_data, cassava_data, cocoa_data, cotton_data, oil_data, rice_data, sorghum_data], axis= 0)

In [3]:
processing_pts = processing_pts.reset_index().drop(["index","Id"],axis=1)
processing_pts.head(3)

,Variable,Status,Commodity,admin1,descript,geometry
0,ploc_au,2.0,aquaculture,Lagos,Potential site,POINT (6.14984 4.60110)
1,ploc_au,2.0,aquaculture,Rivers,Potential site,POINT (6.77950 4.96090)
2,ploc_au,2.0,aquaculture,Bayelsa,Potential site,POINT (3.93104 6.52006)


In [4]:
print(f"There are about {processing_pts.shape[0]} known and recognized food processing points in Nigeria")

There are about 51 known and recognized food processing points in Nigeria


In [5]:
processing_pts.descript.unique()

array(['Potential site', 'Existing Processor', 'Potential State',
       'Existing processor', 'Existing processors', 'Existing Processors'],
      dtype=object)

#### Cleaning

In [6]:
processing_pts["descript"].replace("Potential State","Potential processor", inplace=True)
processing_pts["descript"].replace(["Existing Processor", "Existing processor",
                                    "Existing processors", "Existing Processors"],
                                   "Existing processor", inplace=True)

In [7]:
processing_pts.columns

Index(['Variable', 'Status', 'Commodity', 'admin1', 'descript', 'geometry'], dtype='object')

In [8]:
processing_pts['LATITUDE']  = processing_pts.geometry.y
processing_pts['LONGITUDE'] = processing_pts.geometry.x

In [9]:
my_map = folium.Map(location = [processing_pts['LATITUDE'].mean(), processing_pts['LONGITUDE'].mean()], 
                    zoom_start = 6.2, control_scale = True)

In [10]:
my_map

In [11]:
for index, location_info in processing_pts.iterrows(): 
    folium.Marker([location_info["LATITUDE"], location_info["LONGITUDE"]],
                 popup = "<span style='font-size:1.9rem; color: green;'>%s</span><br><b>Status: </b>%s<br><b>Coordinates: </b>(%f, %f)" %(location_info["Commodity"], location_info["descript"], location_info["LONGITUDE"], location_info["LATITUDE"]), 
                 icon = folium.Icon(color="green")).add_to(my_map)

In [12]:
my_map

#### Adding Layers and Control to the map using folium raster layers

In [13]:
folium.raster_layers.TileLayer('Open Street Map').add_to(my_map)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(my_map)
folium.raster_layers.TileLayer('Stamen Toner').add_to(my_map)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(my_map)

# add layer control to show different maps
folium.LayerControl().add_to(my_map)

In [14]:
my_map

#### Saving the map

In [15]:
my_map.save("results/All food processing points.html")